# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

## Router Chain: 实现条件判断的大模型调用


这段代码构建了一个可定制的链路系统，用户可以提供不同的输入提示，并根据这些提示获取适当的响应。

主要逻辑：从`prompt_infos`创建多个`LLMChain`对象，并将它们保存在一个字典中，然后创建一个默认的`ConversationChain`，最后创建一个带有路由功能的`MultiPromptChain`。

![](../images/router_chain.png)

In [19]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [20]:
physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

biology_template = """你是一位非常聪明的生物学家。你擅长从生物学的角度出发，回答各种生物学相关的问题，
并且能够将复杂的生物学概念解释得通俗易懂。而且还能够将生物学知识与其他学科联系起来，回答更加复杂的问题。
你能够分点回答每个问题。
问题如下:
{input}"""

computer_template = """你是一位非常聪明的计算机科学家。你擅长回答关于计算机科学的问题，
你也参加过一些业界尖端项目，有足够的项目履历和经验，你能够回答各种跟计算机相关的问题，并能够用通俗易懂的方式去回答和解释这些问题，
以及一些计算机概念。
问题如下:
{input}"""

chinese_template = """你是一位非常博学的汉语言文学老师。你擅长能够以直观，生动有趣，通俗易懂的方式去回答汉语言文学相关的问题，
并能够将抽象的概念解释的通俗易懂。你还能够把汉语言文学知识与其他学科联系起来，回答更加复杂的问题。
问题如下:
{input}"""

In [21]:
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
        {
        "name": "生物",
        "description": "适用于回答生物问题",
        "prompt_template": biology_template,
    },
        {
        "name": "计算机",
        "description": "适用于回答计算机领域的问题",
        "prompt_template": computer_template,
    },
        {
        "name": "汉语言文学",
        "description": "适用于回答汉语言文学相关的问题",
        "prompt_template": chinese_template,
    },
]

In [22]:
llm = OpenAI(model_name="gpt-3.5-turbo-instruct",base_url="https://api.xiaoai.plus/v1", api_key="sk-9o1als9Y9It7G1YoF38936F9061b4fE9B7C082Ec4cA83215")

In [23]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

In [24]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

### 使用 LLMRouterChain 实现条件判断调用

这段代码定义了一个chain对象（LLMRouterChain），该对象首先使用router_chain来决定哪个destination_chain应该被执行，如果没有合适的目标链，则默认使用default_chain。

In [25]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [26]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [27]:
print(destinations)

['物理: 适用于回答物理问题', '数学: 适用于回答数学问题', '生物: 适用于回答生物问题', '计算机: 适用于回答计算机领域的问题', '汉语言文学: 适用于回答汉语言文学相关的问题']


In [28]:
print(destinations_str)

物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机领域的问题
汉语言文学: 适用于回答汉语言文学相关的问题


In [29]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [30]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机领域的问题
汉语言文学: 适用于回答汉语言文学相关的问题

<< INPUT >>
{inp

In [31]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [32]:
print(chain.run("黑体辐射是什么？?"))



> Entering new MultiPromptChain chain...
物理: {'input': '什么黑体辐射是？'}
> Finished chain.


黑体辐射是指一个完美吸收一切辐射的理想物体所发出的热辐射。它的辐射能量密度与频率和温度有关，遵循普朗克定律。在经典物理学中，黑体辐射被认为是一个难以解释的现象，直到引入了量子力学才有了合理的解释。


In [33]:
print(
    chain.run(
        "大于40的第一个质数是多少，使得这个质数加一能被3整除？"
    )
)



> Entering new MultiPromptChain chain...
数学: {'input': '41'}
> Finished chain.
 × 59

首先，我们可以将这个问题分解为两个较小的问题，分别是：
40 × 59 和 1 × 59

然后，我们可以使用乘法的交换律来简化这两个问题：
40 × 59 = 4 × 10 × 59 = 4 × 590
1 × 59 = 1 × 9 × 59 = 1 × 531

现在，我们可以将这两个简化后的问题加起来：
4 × 590 + 1 × 531 = 2360 + 531 = 2891

因此，41 × 59 = 2891。

你看，通过分解和简化问题，我们能够更轻松地回答一个复杂的数学问题。继续保持这种方法，你一定会继续成为一位出色的数学家！


In [34]:
router_chain.verbose = True

In [35]:
print(chain.run("黑洞是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
物理: {'input': '黑洞是什么？物理'}
> Finished chain.
教授的回答是：

黑洞是一种极为密集的天体，它的质量非常大，但体积却非常小。它的引力非常强大，连光都无法逃脱。我们目前还无法直接观测到黑洞，但通过间接的证据，我们相信它存在于宇宙中。关于黑洞的本质及其影响还有很多未解之谜，科学家们正在努力研究和探索。


### Homework

#### 扩展 Demo：实现生物、计算机和汉语言文学老师 PromptTemplates 及对应 Chains

In [36]:
print(chain.run("普通细胞和癌细胞有哪些区别?"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
生物: {'input': '普通细胞和癌细胞有哪些区别?'}
> Finished chain.

1. 细胞结构上的区别：普通细胞通常具有完整的细胞结构，包括细胞膜、细胞质、细胞核和细胞器等，而癌细胞往往缺乏这些结构，形态不规则，细胞器数量也不稳定。

2. 细胞生长速度：普通细胞生长速度通常受到机体内控制机制的调节，保持相对稳定的生长速率，而癌细胞则具有无限制的生长能力，可以不受限制地分裂增殖，形成肿瘤。

3. 细胞分化能力：普通细胞可以根据机体需要分化为不同类型的细胞，如肌肉细胞、神经细胞等，而癌


In [37]:
print(chain.run("计算机架构中ARM和x86有什么区别？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
计算机: {'input': '计算机架构中ARM和x86有什么区别？'}
> Finished chain.


ARM和x86都是计算机的处理器架构，它们有以下几个主要区别：

1. 架构设计：ARM架构是由英国公司ARM Holdings开发的，主要用于移动设备和嵌入式系统，而x86架构是由英特尔和AMD等公司开发的，主要用于个人电脑和服务器。

2. 指令集：ARM架构采用精简指令集（RISC）设计，指令数量相对较少，但每条指令的执行速度较快；而x86架构采用复杂指令集（CISC）设计，指令数量较多，但每条指令的执行速度相对较慢。

3. 处理器类型：ARM处理器通常采用低功耗设计，适合用于移动设备和嵌入式系统，而x86处理器通常采用高性能设计


In [38]:
print(chain.run("在三国演义中，是谁下令斩了吕布？为什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
None: {'input': '在三国演义中，是谁下令斩了吕布？为什么？'}
> Finished chain.
 在三国演义中，下令斩了吕布的是曹操。因为吕布曾经背叛了曹操，而且表现出了很大的威胁，曹操为了巩固自己的权力，便下令斩杀了吕布。同时，吕布也是三国时期的一个重要角色，斩杀他也有助于巩固曹操的统治地位。
